# Get Retriver

In [4]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src/utils')
from doc_to_vertex_search import *

In [5]:
parameters = get_me_parameters('../vector_store_me_parameters/ecg_all_me.json')

In [6]:
embeddings = get_embeddings()

In [7]:
db = get_vector_store(parameters, embeddings)

In [16]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)
filters = [Namespace(name="department", allow_tokens=["dpo"]),
           Namespace(name="loader_method", allow_tokens=["unstructured"])
          ]


numeric_filters = [NumericNamespace(name="chunk_size", value_float=2000.0, op="EQUAL")]


In [17]:
retriever = db.as_retriever()

In [18]:
retriever.search_kwargs = {"filter": filters, "numeric_filter": numeric_filters}

In [19]:
q = "Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?"

In [73]:
q2 = "J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?"

In [20]:
retrieved_documents = retriever.invoke(q)

In [74]:
retrieved_documents = retriever.invoke(q2)

## local llm

In [42]:
# https://huggingface.co/blog/not-lain/rag-chatbot-using-llama3

In [2]:
from langchain.llms import HuggingFacePipeline

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# use quantization to lower GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

/home/qianyucazelles/internal_knowledge_chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.55s/it]


In [2]:
model.device

device(type='cuda', index=0)

In [65]:
SYS_PROMPT = """You are an assistant for answering questions.
You are given the extracted parts of several documents rank by relevance with document name and a question. Please pick the most relevant document related to the question then 
provide a conversational answer in the same language as the question. Always cite the referred document in your response.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

In [66]:
def format_prompt(prompt,retrieved_documents):
  """using the retrieved documents we will prompt the model to generate our responses"""
  formatted_prompt = f"Question:{prompt}\nContext: \n"
  separated_line = "-"*50+"\n"
  for idx,doc in enumerate(retrieved_documents) :
    formatted_prompt+= f"{separated_line} Ranked Document: {idx+1} \nName: {doc.metadata['document_name']}\nContent: {doc.page_content} \n"
  return formatted_prompt

In [75]:
formatted_prompt = format_prompt(q2,retrieved_documents)

In [76]:
messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]

In [69]:
from transformers import TextIteratorStreamer

from threading import Thread

In [77]:
# set streamer
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


# apply chat template
messages_tmpl = tokenizer.apply_chat_template(messages,
                       tokenize=False,
                       add_generation_prompt=True
                      )

# tokenize msgs
inputs = tokenizer([messages_tmpl], return_tensors="pt").to('cuda')

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
generation_kwargs = dict(inputs, 
                         streamer=streamer, 
                         max_new_tokens=1500,
                         eos_token_id=terminators)
# chat
thread = Thread(target=model.generate,
                kwargs=generation_kwargs)
thread.start() 

generated_text = ''
for _, new_text in enumerate(streamer):
    generated_text += new_text


In [78]:
q2

"J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?"

In [79]:
generated_text

"Bonjour! Je comprends que vous avez des difficultés à comprendre les différentes obligations que vous avez vis-à-vis des sous-traitants qui travaillent pour vous.\n\nSelon le chapitre IV du Règlement européen sur la protection des données, un sous-traitant doit informer immédiatement le responsable du traitement si, selon lui, une instruction constitue une violation du présent règlement ou d'autres dispositions du droit de l'Union ou du droit des États membres relatives à la protection des données (Article 28, h).\n\nEn outre, le sous-traitant doit prendre des garanties suffisantes pour la mise en œuvre de mesures techniques et organisationnelles appropriées pour que le traitement réponde aux exigences du présent règlement. Si l'autre sous-traitant ne remplit pas ses obligations en matière de protection des données, le sous-traitant initial demeure pleinement responsable devant le responsable du traitement de l'exécution par l'autre sous-traitant de ses obligations (Article 28, 4).\n\